In [175]:
import subprocess
import xml.etree.ElementTree as ET

In [176]:
# Read Java file and srcML it to XML
input_file = 'Input.java'
with open('Input.xml', 'w+') as f:
    subprocess.run(['srcml', input_file], stdout=f)

In [177]:
# Parse XML and define namespace
tree = ET.parse('Input.xml')
root = tree.getroot()
ns = {'src': 'http://www.srcML.org/srcML/src'}

In [178]:
# Return an Element of println statement
def println_elem(x, tail='\n'):
    expr_stmt = ET.Element('expr_stmt')
    expr_stmt.tail = tail
    expr = ET.SubElement(expr_stmt, 'expr')
    expr.tail = ';'
    call = ET.SubElement(expr, 'call')
    
    name = ET.SubElement(call, 'name')
    name1 = ET.SubElement(name, 'name')
    name1.text = 'System'
    operator1 = ET.SubElement(name, 'operator')
    operator1.text = '.'
    name2 = ET.SubElement(name, 'name')
    name2.text = 'out'
    operator2 = ET.SubElement(name, 'operator')
    operator2.text = '.'
    name3 = ET.SubElement(name, 'name')
    name3.text = 'println'
    
    argument_list = ET.SubElement(call, 'argument_list')
    argument_list.text = '('
    argument = ET.SubElement(argument_list, 'argument')
    argument.tail = ')'
    
    expr1 = ET.SubElement(argument, 'expr')
    literal = ET.SubElement(expr1, 'literal', type="string")
    literal.text = ''.join(['\"', x, ' = \"'])
    operator3 = ET.SubElement(expr1, 'operator')
    operator3.text = ' + '
    name4 = ET.SubElement(expr1, 'name')
    name4.text = x
    
    return expr_stmt # System.out.println(“x = ” + x);

# Case 1: Declaration with initialisation
* int a = 0, b, c = 1;

In [179]:
for elem in root.iter():   
    for decl_stmt in elem.findall('src:decl_stmt', ns):
        offset = 1
        
        var_list = []
        decls = decl_stmt.findall('src:decl', ns)
        
        for decl in decls:
            if decl.find('src:init', ns):
                var_list.append(decl.find('src:name', ns).text)
        
        for var in var_list:
            elem.insert(offset + list(elem).index(decl_stmt), println_elem(var, tail=decl_stmt.tail))
            offset += 1

# Case 2: Expression with assignments

Below are statements that contain expressions (expr), where there are potential assignments, according to srcML's Java Documentation [1].

* expr_stmt (expr_stmt)

* assert statement (assert)

* case statement (case)

* return statement (return)

In these statements, there are two possible cases of assignment:

* Subcase 1: left value of operators '=', '+=', '-=', '++', or '--' (e.g. d = e = 2; f += 3;)

* Subcase 2: right value of operators '++' or '--' (e.g. --g;)

## References

[1] srcML v1.0.0 Java Documentation. https://www.srcml.org/doc/java_srcML.html. (Accessed July 2020)

In [180]:
for elem in root.iter():
    for exprs in elem.findall('src:expr_stmt', ns) + elem.findall('src:assert', ns) \
        + elem.findall('src:case', ns) + elem.findall('src:return', ns):
        offset = 1
        
        var_list = []
        expr = exprs.find('src:expr', ns)
        
        expr_list = list(expr)
        names = [expr_list.index(name) for name in expr.findall('src:name', ns)]
        operators = [expr_list.index(operator) for operator in expr.findall('src:operator', ns)]
            
        for i in range(len(expr_list)):
            # Subcase 1 and 2
            if i in names and i + 1 in operators \
                and expr_list[i + 1].text in ['=', '+=', '-=', '++', '--'] \
                or i in names and i - 1 in operators \
                and expr_list[i - 1].text in ['++', '--']:
                var_list.append(expr_list[i].text)
        
        for var in var_list:
            elem.insert(offset + list(elem).index(exprs), println_elem(var, tail=exprs.tail))
            offset += 1

In [181]:
# Write back to XML file and convert back to Java
tree.write('Output.xml')
with open('Output.java', 'w+') as f:
    subprocess.run(['srcml', 'Output.xml'], stdout=f)

# Other possible cases

## Case 3: Loop condition initialisation

* while(--g) {...}
* for(int i = 0; i < 10; ++i) {...}

## Case 4: ...

In [ ]:
# Appendix: Original implementation of case 2
# 
# for elem in root.iter():
#     for expr_stmt in elem.findall('src:expr_stmt', ns):
#         offset = 1
        
#         var_list = []
#         expr = expr_stmt.find('src:expr', ns)
        
#         expr_list = list(expr)
#         names = [expr_list.index(name) for name in expr.findall('src:name', ns)]
#         operators = [expr_list.index(operator) for operator in expr.findall('src:operator', ns)]
            
#         for i in range(len(expr_list)):
#             # Subcase 1 and 2
#             if i in names and i + 1 in operators \
#                 and expr_list[i + 1].text in ['=', '+=', '-=', '++', '--'] \
#                 or i in names and i - 1 in operators \
#                 and expr_list[i - 1].text in ['++', '--']:
#                 var_list.append(expr_list[i].text)
        
#         for var in var_list:
#             elem.insert(offset + list(elem).index(expr_stmt), println_elem(var, tail=expr_stmt.tail))
#             offset += 1